# 실험 준비

In [40]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [41]:
from src.preprocess import split_train_valid_test_tabular

In [42]:
import os

os.listdir('./run_result_tabular/')

['result_abalone9-18.csv',
 'result_shuttle-c0-vs-c4.csv',
 'result_vowel0.csv',
 'result_wine.csv',
 'result_yeast-1-2-8-9_vs_7.csv',
 'result_yeast4.csv',
 'result_yeast5.csv',
 'result_yeast6.csv']

In [74]:
data_name = 'yeast6'

In [75]:
(
    _,
    _,
    _,
    _,
    _,
    test_y,
) = split_train_valid_test_tabular('./tabular_data/', data_name, 0.7)

c:\Users\ghktj\Desktop\my-paper\src\preprocess.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["label"] = y_test


In [76]:
df = pd.read_csv('./run_result_tabular/result_' + data_name + '.csv')
cols = ['trainer_name', 'sampling_term','sampling_ratio','initial_epoch']
df.groupby(cols)[['roc_auc','pr_auc']].agg(['mean','std'])

roc_auc      pr_auc  \
                                                           mean std    mean   
trainer_name sampling_term sampling_ratio initial_epoch                       
BaseTrainer  0             0.00           0              0.6537 NaN  0.1891   
NewTrainer   1             0.01           5              0.7147 NaN  0.2513   
                                          20             0.6949 NaN  0.2267   
                           0.10           5              0.7203 NaN  0.2882   
                                          20             0.7393 NaN  0.2478   
             5             0.01           5              0.6069 NaN  0.1798   
                                          20             0.6678 NaN  0.2133   
                           0.10           5              0.6976 NaN  0.2381   
                                          20             0.7018 NaN  0.2362   

                                                             
                                                        std  
trainer_name sampling_term sampling_ratio initial_epoch      
BaseTrainer  0             0.00           0             NaN  
NewTrainer   1             0.01           5             NaN  
                                          20            NaN  
                           0.10           5             NaN  
                                          20            NaN  
             5             0.01           5             NaN  
                                          20            NaN  
                           0.10           5             NaN  
                                          20            NaN

In [77]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

In [78]:
anomaly_score = np.zeros((len(test_y), 8))
result_df = pd.DataFrame(columns=['mean','median','max','min'], index=['roc_auc','pr_auc'])

PATH = './ensemble_tabular_3/'
idx = 0
for data in os.listdir(PATH):
    if data_name in data:
        with open(PATH + data, 'rb') as f:
            tmp = pickle.load(f)
            anomaly_score[:, idx] = tmp
            idx += 1

tst_ano_scr_med = np.median(anomaly_score, axis=1)
tst_ano_scr_mean = np.mean(anomaly_score, axis=1)
tst_ano_scr_max = np.max(anomaly_score, axis=1)
tst_ano_scr_min = np.min(anomaly_score, axis=1)

idx = 0
for tst_ano_scr in [tst_ano_scr_mean, tst_ano_scr_med, tst_ano_scr_max, tst_ano_scr_min]:
    roc_auc = roc_auc_score(test_y, tst_ano_scr)
    _precision, _recall, _ = precision_recall_curve(test_y, tst_ano_scr)
    pr_auc = auc(_recall, _precision)
    result_df.iloc[0, idx] = roc_auc
    result_df.iloc[1, idx] = pr_auc
    idx += 1

result_df
# if config.data == "tabular":
#     tst_y_true = total_y[-len(tst_ano_scr):]
# else:
    # tst_y_true = total_y[tst_start_idx : len(window_anomaly_score_result)] # 여기 time_series 계산


,mean,median,max,min
roc_auc,0.698558,0.694626,0.716776,0.699607
pr_auc,0.238208,0.227948,0.242418,0.31622


# time

In [82]:
from src.preprocess import split_train_valid_test

In [79]:
import os

os.listdir('./time_series_data')

['137_UCR_Anomaly_InternalBleeding18_2300_4485_4587.txt',
 '139_UCR_Anomaly_InternalBleeding20_2700_5759_5919.txt',
 '141_UCR_Anomaly_InternalBleeding5_4000_6200_6370.txt',
 '143_UCR_Anomaly_InternalBleeding8_2500_5865_5974.txt']

In [90]:
data_name = '141_UCR_Anomaly_InternalBleeding5_4000_6200_6370'

In [88]:
_, _, test_x, _, _, _ = split_train_valid_test(
    './time_series_data/', data_name, 0.7
)

data_len = test_x.shape[0] - 60 + 1

In [91]:
df = pd.read_csv('./run_result_time/result_' + data_name + '.csv')
cols = ['trainer_name', 'sampling_term','sampling_ratio','initial_epoch']
df.groupby(cols)[['roc_auc','pr_auc']].agg(['mean','std'])

roc_auc      pr_auc  \
                                                           mean std    mean   
trainer_name sampling_term sampling_ratio initial_epoch                       
BaseTrainer  0             0.00           0              0.5505 NaN  0.1059   
NewTrainer   1             0.01           5              0.6227 NaN  0.1517   
                                          20             0.5221 NaN  0.0874   
                           0.10           5              0.5925 NaN  0.1637   
                                          20             0.5843 NaN  0.1049   
             5             0.01           5              0.5298 NaN  0.1036   
                                          20             0.5994 NaN  0.1204   
                           0.10           5              0.6126 NaN  0.1668   
                                          20             0.5803 NaN  0.1036   

                                                             
                                                        std  
trainer_name sampling_term sampling_ratio initial_epoch      
BaseTrainer  0             0.00           0             NaN  
NewTrainer   1             0.01           5             NaN  
                                          20            NaN  
                           0.10           5             NaN  
                                          20            NaN  
             5             0.01           5             NaN  
                                          20            NaN  
                           0.10           5             NaN  
                                          20            NaN

In [ ]:
anomaly_score = np.zeros((len(test_y), 8))
result_df = pd.DataFrame(columns=['mean','median','max','min'], index=['roc_auc','pr_auc'])

PATH = './ensemble_tabular_1/'
idx = 0
for data in os.listdir(PATH):
    if data_name in data:
        with open(PATH + data, 'rb') as f:
            tmp = pickle.load(f)
            anomaly_score[:, idx] = tmp
            idx += 1

tst_ano_scr_med = np.median(anomaly_score, axis=1)
tst_ano_scr_mean = np.mean(anomaly_score, axis=1)
tst_ano_scr_max = np.max(anomaly_score, axis=1)
tst_ano_scr_min = np.min(anomaly_score, axis=1)

idx = 0
for tst_ano_scr in [tst_ano_scr_mean, tst_ano_scr_med, tst_ano_scr_max, tst_ano_scr_min]:
    roc_auc = roc_auc_score(test_y, tst_ano_scr)
    _precision, _recall, _ = precision_recall_curve(test_y, tst_ano_scr)
    pr_auc = auc(_recall, _precision)
    result_df.iloc[0, idx] = roc_auc
    result_df.iloc[1, idx] = pr_auc
    idx += 1

result_df
# if config.data == "tabular":
#     tst_y_true = total_y[-len(tst_ano_scr):]
# else:
    # tst_y_true = total_y[tst_start_idx : len(window_anomaly_score_result)] # 여기 time_series 계산
